In [1]:
import sys,time
from pathlib import Path 
BASE_PATH = 'D:/jupyter_notebooks/Arabidopsis_seed_detection'
sys.path.append(BASE_PATH+"\\research\\")
sys.path.append(BASE_PATH+"\\research\\object_detection\\")
sys.path.append(BASE_PATH+"\\research\\object_detection\\utils\\")
sys.path.append(BASE_PATH+"\\research\\slim\\")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import numpy as np
import os
import six.moves.urllib as urllib
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from PIL import Image
from matplotlib import pyplot as plt
print(tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
from tensorflow.python.client import device_lib

config = tf.ConfigProto(log_device_placement=True)

#test images dir
#PATH_TO_TEST_IMAGES_DIR = BASE_PATH+"\\test_images"
PATH_TO_TEST_IMAGES_DIR = "D:\\work\\MSU\\tensorflow_object_detection_arabidopsis_seeds\\05_comparation_between_ImageJ_Faster_RCNN\\ImageJ_simple_seed_scans_for_Fanrui\\split_1_10_3_4_5\\split_into_plate"
TEST_IMAGE_PATHS=[]
for root, dirs, files in os.walk(PATH_TO_TEST_IMAGES_DIR):
        for f in files:
            if not f.endswith("csv"):
                TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR,f))    
PATH_TO_CKPT=BASE_PATH+"\\seed_graph_train\\frozen_inference_graph.pb"
print(PATH_TO_CKPT)
PATH_TO_LABELS = BASE_PATH+"\\mscoco_label_map.pbtxt"
print("ok")
NUM_CLASSES = 1
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=False)
category_index = label_map_util.create_category_index(categories)

print("ok")
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
print("ok")
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      output_dict = sess.run(tensor_dict,feed_dict={image_tensor: np.expand_dims(image, 0)})
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      #print(output_dict['detection_boxes'])
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict
output = open(PATH_TO_TEST_IMAGES_DIR+"/Seed_count_results_"+time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())+".csv","a")
for image_path in TEST_IMAGE_PATHS:
  print("Start: ",time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime()))
  image = Image.open(image_path)
  (im_width, im_height) = image.size
  image_np = load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  scores = output_dict["detection_scores"]
  count = len(scores[scores>=0.5])
  output.write("%s,%d\n"%(image.filename.split("\\")[-1],count))
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],      
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=2,
      max_boxes_to_draw=None,
      min_score_thresh=.5,
      skip_scores=True,
      skip_labels=True)
  plt.figure(figsize=((float(format(float(im_width)/float(200),'.2f')), float(format(float(im_height)/float(200),'.2f')))))
  plt.imshow(image_np)
  plt.gca().xaxis.set_major_locator(plt.NullLocator())
  plt.gca().yaxis.set_major_locator(plt.NullLocator())
  plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
  #plt.axis('off') 
  plt.margins(0,0)
  plt.savefig(image_path+"_"+str(count)+"_count.jpg")
  print(image.filename,count)
  print("End: ",time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime()))

C:\Users\Fanrui Meng\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Fanrui Meng\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Fanrui Meng\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Fanrui Meng\AppData\Roami

1.13.2
D:/jupyter_notebooks/Arabidopsis_seed_detection\seed_graph_train\frozen_inference_graph.pb
ok
ok
ok
Start:  2021-06-09_13-48-22
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan10_032018217.jpg_0.jpg 149
End:  2021-06-09_13-49-57
Start:  2021-06-09_13-49-57
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan10_032018217.jpg_1.jpg 60
End:  2021-06-09_13-51-35
Start:  2021-06-09_13-51-35
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan10_032018217.jpg_10.jpg 96
End:  2021-06-09_13-53-22
Start:  2021-06-09_13-53-22
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\I

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan1_031418208.jpg_6.jpg 73
End:  2021-06-09_14-23-24
Start:  2021-06-09_14-23-24
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan1_031418208.jpg_7.jpg 120
End:  2021-06-09_14-25-04
Start:  2021-06-09_14-25-04
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan1_031418208.jpg_8.jpg 86
End:  2021-06-09_14-26-33
Start:  2021-06-09_14-26-33
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan1_031418208.jpg_9.jpg 94
End:  2021-06-09_14-28-06
Start:  2021-06

D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan5_031418212.jpg_3.jpg 57
End:  2021-06-09_15-17-15
Start:  2021-06-09_15-17-15
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan5_031418212.jpg_4.jpg 55
End:  2021-06-09_15-18-54
Start:  2021-06-09_15-18-54
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan5_031418212.jpg_5.jpg 76
End:  2021-06-09_15-20-31
Start:  2021-06-09_15-20-31
D:\work\MSU\tensorflow_object_detection_arabidopsis_seeds\05_comparation_between_ImageJ_Faster_RCNN\ImageJ_simple_seed_scans_for_Fanrui\split_1_10_3_4_5\split_into_plate\scan5_031418212.jpg_6.jpg 54
End:  2021-06-09_15-22-06
Start:  2021-06-